# 自定义ipywidgets控件

In [1]:
%%javascript
require(["./gauge.min.js"], function(){
    
});

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as widgets
from traitlets import Float, Unicode, Int

In [3]:
class GuageWidget(widgets.DOMWidget):
    _view_module = Unicode('guage').tag(sync=True)    
    _view_name = Unicode('GuageView').tag(sync=True)
    size = Int(300).tag(sync=True)
    title = Unicode().tag(sync=True)
    unit = Unicode().tag(sync=True)
    value = Float().tag(sync=True)

In [4]:
%%javascript
require.undef("guage");
define('guage', ["jupyter-js-widgets"], function(widgets){
    var GuageView = widgets.DOMWidgetView.extend({
        render: function(){
            var size = this.model.get('size');
            console.log("render");
            this.$canvas = $('<canvas></canvas>').appendTo(this.$el);
            
            this.$canvas.attr('width', size);
            this.$canvas.attr('height', size);
            console.log(this.$canvas[0]);
            var gauge = new Gauge({
                renderTo  : this.$canvas[0],
                width     : size,
                height    : size,
                glow      : true,
                units     : this.model.get('unit'),
                title     : this.model.get('title')
            });
            this.gauge = gauge;
            setTimeout(function(){gauge.draw();}, 100);            
        },
        
        update: function(){
            console.log("update");
            this.gauge.setValue(this.model.get('value'));
            return GuageView.__super__.update.apply(this);             
        }
    });
    
    return {GuageView: GuageView};
});

<IPython.core.display.Javascript object>

In [5]:
guage = GuageWidget(title="Velocity", size=300)
guage

In [26]:
from IPython.core.magic import register_cell_magic, register_line_cell_magic
    
@register_line_cell_magic
def cell_call(*args):
    shell = get_ipython()
    global_name = "_registered_cells"
    if len(args) == 2:
        line, cell = args
    else:
        line, = args
        cell = None

    name = line.strip()
    if cell:
        cells = shell.user_ns.get(global_name, {})
        if not cells:
            shell.user_ns[global_name] = cells
        cells[name] = cell
    else:
        cell = shell.user_ns.get(global_name, {}).get(name, None)
        if cell is not None:
            shell.run_cell(cell)

In [6]:
%load_ext py2js

In [7]:
%%py2js
require.undef("guage")

def main(widgets):
    def render(self):
        window.tmp = self
        size = self.model.get('size')
        canvas = self.canvas = jQuery('<canvas></canvas>').appendTo(self["$el"])
        for name in ('width', 'height'):
            canvas.attr(name, size)

        self.gauge = gauge = Gauge({
            "renderTo": self.canvas[0],
            "width": size,
            "height": size,
            "glow": True,
            "units": self.model.get("unit"),
            "title": self.model.get("title"),
        })
        window.setTimeout(lambda : gauge.draw(), 100)
        
    def update(self):
        self.gauge.setValue(self.model.get('value'))
        return GuageView.__super__.update.apply(self)
        
    GuageView = widgets.DOMWidgetView.extend(dict(
        render=render,
        update=update))
    return dict(GuageView=GuageView)

define('guage', ['jupyter-js-widgets'], main)

In [8]:
guage = GuageWidget(title="Velocity", size=300)
guage

In [9]:
guage.value = 20

In [30]:
import zmq
import json
from zmq.eventloop.zmqstream import ZMQStream

class Messager:
    def __init__(self, callback, ip="127.0.0.1", port="2223"):
        context = zmq.Context.instance()
        self.callback = callback
        self.socket = context.socket(zmq.SUB)
        self.socket.setsockopt(zmq.SUBSCRIBE, b"")
        self.socket.bind("tcp://{}:{}".format(ip, port))
        self.stream = ZMQStream(self.socket)
        self.stream.on_recv(self.get_message) 
        
    def get_message(self, msg):
        data = json.loads(b"".join(msg).decode("utf-8"))
        self.callback(data)

    def close(self):
        self.stream.close()
        self.socket.close()
        
    def __del__(self):
        self.close()

In [35]:
%%cell_call send_values
%%python --bg
import zmq
import time
import json
import random
context = zmq.Context()

socket = context.socket(zmq.PUB)
socket.connect("tcp://127.0.0.1:2223")

model = dict(value=0)

for i in range(200):
    model["value"] = model["value"] + random.uniform(-0.5, 1)
    socket.send_string(json.dumps(model))
    time.sleep(0.02)

In [32]:
from ipywidgets import Button, VBox, HBox, Checkbox

start_button = Button(description="Start")
run_button = Button(description="Run")
stop_button = Button(description="Stop")

def start_callback(event):
    global messager
    messager = Messager(lambda data:setattr(guage, "value", data["value"]))

def run_callback(event):
    from IPython.display import clear_output
    clear_output()
    cell_call("send_values")
    
def stop_callback(event):
    messager.close()
    
start_button.on_click(start_callback)
run_button.on_click(run_callback)
stop_button.on_click(stop_callback)

HBox([VBox([start_button, run_button, stop_button]), guage])

Starting job # 27 in a separate thread.
